## Bu ipynb dosyasında Positionel Encoder ve Embedding katmanlarını inceleyeceğiz.Bu katmanları daha iyi hale getirmeye çalışacağız.Ama tüm bu işlemlerden önce tokenizer kodlarını yazalım.Yazdıktan sonra PE ve Embedding kavramlarının detayına inmeye devam edelim.

----
-----

### ÖNCE TOKENİZER

In [1]:
import io, re, unicodedata, sentencepiece as spm
import pandas as pd, tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd


In [2]:
df = pd.read_csv(r"C:\Users\hdgn5\OneDrive\Masaüstü\Transformerlar\Konu Anlatımları\Encoder - Decoder - PE - ATTN ( İLERİ DÜZEY ) = TF\(2) -- PE & Embedding\örnek_set.csv")
df.head()

,input,output
0,Merhaba,"Merhaba, size nasıl yardımcı olabilirim?"
1,Nasılsın?,"İyiyim, teşekkür ederim. Siz nasılsınız?"
2,Adın ne?,Ben bir yapay zekâ asistanıyım. Adım yok ama y...
3,Kaç yaşındasın?,"Benim yaşım yok, dijitalim!"
4,Bugün günlerden ne?,"Maalesef tarih bilgim yok, ama sistem saatinde..."


In [3]:
raw_inputs = df["input"].astype(str).to_list()
raw_targets = df["output"].astype(str).to_list()

In [4]:
def clean_texts(text:str) -> str:
    text = unicodedata.normalize("NFC",text).lower()
    text = re.sub(r"[^a-zçğıöşüğ0-9\s]", " ", text)
    return re.sub(r"\s+", " ", text).strip()

In [5]:
USER, BOT , CASUAL = "<user>" , "<bot>" , "<casual>"

input_texts = [f"{USER} {CASUAL} {clean_texts(t)}"  for t in raw_inputs]
target_texts = [f"{BOT} {CASUAL} {clean_texts(t)}" for t in raw_targets]

In [6]:
model_prefix , vocab_desired , vocab_limit = "bpe_tr" , 8000 , 1000 

with io.open("corpus.txt" , "w" , encoding="utf-8") as f :
    f.write("\n".join(input_texts + target_texts))

In [7]:
spm.SentencePieceTrainer.Train(
    input = "corpus.txt",
    model_prefix = model_prefix , 
    model_type = "bpe" , 
    vocab_size = min(vocab_desired,vocab_limit) , 
    user_defined_symbols= [
        "<pad>", "<sos>" , "<eos>" , 
        BOT  , CASUAL , USER
        ]
    
    )

In [8]:
sp = spm.SentencePieceProcessor(model_file=f"{model_prefix}.model")
pad_id = sp.piece_to_id("<pad>")
sos_id = sp.piece_to_id("<sos>")
eos_id = sp.piece_to_id("<eos>")
unk_id = sp.unk_id()
vocab_size = sp.get_piece_size()

In [9]:
enc = [sp.encode(t , out_type = int)  for t in input_texts]
din = [[sos_id] + sp.encode(t,out_type = int) for t in target_texts]
dec = [sp.encode(t,out_type=int) + [eos_id] for t in target_texts]

In [10]:
max_len = max(max(map(len,dec)) , max(map(len,din)) , max(map(len,dec)))
print(max_len)

16


In [11]:
enc_in = pad_sequences(enc,maxlen=max_len ,padding="post" , value=pad_id)
dec_in = pad_sequences(din,maxlen=max_len , padding="post" , value=pad_id)
dec_out = pad_sequences(dec ,maxlen=max_len,padding="post" , value=pad_id)

In [12]:
drop_rate = 0.1

def unk_dropout(inputs,targets,rate=drop_rate,unk=unk_id):
    e = inputs["encoder_input"]
    d = inputs["decoder_input"]

    mask_e = tf.cast(tf.random.uniform(tf.shape(e)) < rate , e.dtype)
    mask_d = tf.cast(tf.random.uniform(tf.shape(d)) <rate , d.dtype)

    inputs["encoder_inputs"] = tf.where(mask_e ==1 , unk  , e)
    inputs["decoder_inputs"] = tf.where(mask_d == 1 ,unk , d)

    return inputs,targets

In [13]:
def schedule(ep):
    if ep < 5 : return 0.05
    if ep < 15 :  return 0.12

    return 0.08

class UnkSchedule(tf.keras.callbacks.Callback):
    def en_epoch_begin(self,epoch,logs = None):
        rate = schedule(epoch)
        drop_rate.assign(rate)
        print(f"[UNK-Dropout] epoch {epoch} → rate={rate:.2f}")

In [14]:
BATCH_SIZE  = 32    # her adımda GPU’ya gidecek örnek sayısı
BUFFER_SIZE = 1000  # shuffle havuzu (rastgelelik için)

dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            {
                "encoder_input": enc_in,   # ← modelin encoder girdisi
                "decoder_input": dec_in    # ← decoder girdisi (<sos> + hedef)
            },
            dec_out                       # ← decoder hedef dizisi (<eos> ekli)
        )
    )
    # 1️⃣  Rastgele %10 token’ı <unk> yap → dayanıklılık artır
    .map(unk_dropout, num_parallel_calls=tf.data.AUTOTUNE)

    # 2️⃣  BUFFER_SIZE kadar örnek bellekte karıştır → her epoch farklı sıra
    .shuffle(BUFFER_SIZE)

    # 3️⃣  Sabit boyutlu kümeler oluştur; eksik son batch’i at
    .batch(BATCH_SIZE, drop_remainder=True)

    # 4️⃣  Eğitim adımı GPU’da çalışırken bir sonraki batch’i hazırla
    .prefetch(tf.data.AUTOTUNE)
)

In [15]:
# --------------------- Yüzdesel Bilgi Log’u ----------------------------------
total_tokens   = (enc_in != pad_id).sum() + (dec_in != pad_id).sum()
unk_tokens_est = int(total_tokens * drop_rate)    # teorik beklenti
print(f"UNK-Dropout Rate: {drop_rate*100:.0f}% → ~{unk_tokens_est:,} token her epoch’ta <unk> olacak")

print("Dataset hazır →", next(iter(dataset.take(1)))[0]["encoder_input"].shape)
print(f"pad:{pad_id}  sos:{sos_id}  eos:{eos_id}  unk:{unk_id}  vocab:{vocab_size}")
print("max_len:", max_len)

UNK-Dropout Rate: 10% → ~92 token her epoch’ta <unk> olacak
Dataset hazır → (32, 16)
pad:3  sos:4  eos:5  unk:0  vocab:1000
max_len: 16


---
----

### Şimdi PE ve Embedding e geçebiliriz. Önce minik bir teorik anlatımı yapalım sonrasında da kod olarak içerisine dalalım.Yukarıda bulunan tokenizer ın üstüne koya koya işleri halletmeye çalışalım.Dosyaları kontrol etmeyi unutmayın

----
---

## 🔹 Embedding Katmanı — Ne Yapar?

- **Giriş**  
  Tokenizer’dan gelen **tamsayı kimlikleri** (token ID’leri).

- **İşlem**  
  Her kimliği, öğrenilebilir bir tabloya (embedding matrisi) bakarak
  **yoğun (float) vektöre** dönüştürür.  
  Matris boyutu **\(V \times d_{\text{model}}\)**  
  \(\;\,V\): sözlük büyüklüğü | \(d_{\text{model}}\): vektör boyutu

- **Çıkış**  
  Dizi uzunluğu \(L\) ise tensör şekli  
  **\((\text{batch\_size},\, L,\, d_{\text{model}})\)**

- **Neden Önemli?**  
  - Benzer anlamlı token’ların vektörleri **yakınlaşır**, farklılar uzaklaşır.  
  - Sayısal (ID) dizisini, sinir ağının işleyebileceği **sürekli vektör** dizisine çevirir.

- **Positional Encoding**  
  Embedding konum bilgisi taşımaz; **sıra bilgisini** enjekte etmek için
  bu vektörlere positional encoding **toplanır**.


---

### Öncelikle token embedding fonksiyonunu tanımlıcaz.Sonrasında ise positionel encoding i tanımlıcaz ve ikisini beraber geliştireceğiz.Gidişatı iyi anlamak için ayrı ayrı modüllerde birleştireceğiz.

---

# 1️⃣ Temel Token-Embedding (Keras Tokenizer’a göre)

* Transformer’da ilk adım: dizideki her token_id → sabit boyutlu vektör
(gömme/embedding). Aşağıdaki sınıf bu işi yaparken

* √d_model ölçeklemesi

* pad> vektörlerini sıfırlama

* otomatik maske üretme
sağlar.

In [16]:
import tensorflow as tf
import math

In [ ]:
class TokenEmbedding(tf.keras.layers.Layer):
    def __init__(self, vocab_size, d_model, pad_id=0, **kw):
        super().__init__(**kw)
        self.pad_id = pad_id                      # <pad> id'si
        self.scale  = math.sqrt(d_model)          # √d_model
        self.table  = tf.keras.layers.Embedding(  # V × D matris
            vocab_size, d_model)

    def compute_mask(self,ids,mask=None):
        return tf.not_equal(ids , self.pad_id)
    
    def call(self, ids):                   # ids: (batch, seq_len)
        x = self.table(ids) * self.scale   # ① lookup + √d_model ölçekle
        x *= tf.cast(tf.not_equal(ids, self.pad_id)[..., None], x.dtype)
                                           # ② <pad> satırlarını sıfırla
        return x                           # ③ çıktı: (batch, seq_len, d_model)


### `compute_mask()` tam olarak ne yapıyor?

| Adım | Açıklama |
|------|----------|
| **Fonksiyon İmzası**<br>`compute_mask(self, ids, mask=None)` | *Keras*, katmana bir tensör gönderdiğinde otomatik olarak bu metodu çağırır.<br>• `ids` → `(batch, seq_len)` boyutunda **token-id** dizisi<br>• `mask=None` → Üst katmandan gelen bir maske varsa (`None` yerine tensör) kullanabilirdik; ama burada **yok sayıyoruz** ve kendi maskemizi oluşturuyoruz. |
| **Maske Oluşturma**<br>`tf.not_equal(ids, self.pad_id)` | Dönüş değeri boolean tensörüdür.<br>`True`  →  gerçek token<br>`False` →  `<pad>` token (yani yoksayılacak) |
| **Maskenin Devri** | Bu bool tensör, Keras’ın otomatik “mask zinciri” sayesinde üst katmanlara iletilir.<br>Örneğin `MultiHeadAttention` pad konumlarını görmezden gelir. |
| **Doğru `pad_id` Şartı** | `pad_id` gerçekten PAD’i temsil etmiyorsa ⚠️ maskede hatalı sonuç çıkar → Model PAD’leri “gerçek” sanır. |

> **Özet:** `compute_mask()` dışarıdan maske almıyor (`None`), onun yerine **`pad_id`**’ye bakarak **yeni bir maske** oluşturup Keras’a geri veriyor. Bu mekanizma yalnızca `pad_id` doğru ayarlanmışsa işe yarar.


#### `call()` – Ne yapar?

| Aşama | Kod | Ne olur? |
|-------|-----|----------|
| 1. **Lookup** | `x = self.table(ids)` | `(B, L)` token-ID → `(B, L, D)` vektör |
| 2. **Ölçekle** | `* self.scale` | Tüm vektörleri √d_model ile çarp |
| 3. **PAD’leri sıfırla** | `x *= tf.cast(ids!=pad_id, x.dtype)[...,None]` | `<pad>` vektörleri 0 yapılır |
| 4. **Dön** | `return x` | Çıktı: `(B, L, D)` embed tensörü |

> **Kısaca:** lookup → √d_model → pad sıfırla → embed edilmiş çıktı.
